In [1]:
!pip install torchinfo


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import torch
import sys
import os
from torchinfo import summary

# Ajouter le chemin pour importer ResNet18
sys.path.append(os.path.abspath("../LAB1"))
from resnet import ResNet18

def calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True):
    # Créer le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet18().to(device)
    checkpoint = torch.load("./pruned_model.pth", map_location=device)
    checkpoint = torch.load("./pruned_model.pth", map_location=device)
    model.load_state_dict(checkpoint, strict=False) 
    print("Modèle chargé en half precision !")
    
    # Afficher les statistiques du modèle
    stats = summary(model, (1, 3, 32, 32), verbose=0)
    
    total_params = stats.total_params
    total_macs = stats.total_mult_adds  # Nombre d'opérations MAC (approximation des FLOPs)
    
    # Convertir MAC en FLOPs (environ 2 FLOPs par MAC)
    total_flops = total_macs * 2
    
    # Calculer le score
    p_s = structured_pruning
    p_u = global_pruning
    q_w = 16 if is_half_precision else 32  # 16 bits pour half precision, 32 pour full
    
    # On suppose q_a = 32 bits pour les activations
    max_q = max(q_w, 16)
    
    w = total_params
    f = total_flops
    
    param_term = (1 - (p_s + p_u)) * (q_w/32) * w / (5.6e6)
    ops_term = (1 - p_s) * (max_q/32) * f / (8.3e8)
    
    score = param_term + ops_term
    
    print(f"Nombre total de paramètres: {total_params:,}")
    print(f"Nombre total d'opérations (FLOPs): {total_flops:,}")
    print(f"Score calculé: {score:.4f}")
    
    return score

# Calculer le score avec vos paramètres
score = calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True)

C:\Users\leami\AppData\Local\Temp\ipykernel_11700\643435494.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./pruned_model.pth", map_location=d

Modèle chargé en half precision !
Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 1.1340


C:\Users\leami\AppData\Local\Temp\ipykernel_11700\643435494.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./pruned_model.pth", map_location=d

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load("./pruned_model.pth", map_location=device)
print(checkpoint.keys())  # Vérifie les clés contenues dans le fichier


odict_keys(['conv1.weight_orig', 'conv1.weight_mask', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight_orig', 'layer1.0.conv1.weight_mask', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight_orig', 'layer1.0.conv2.weight_mask', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight_orig', 'layer1.1.conv1.weight_mask', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight_orig', 'layer1.1.conv2.weight_mask', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight_orig', 'layer2.0.conv1.weight_mask', 'layer

C:\Users\leami\AppData\Local\Temp\ipykernel_11700\3176181146.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./pruned_model.pth", map_location=d

Modele de base

In [3]:
import torch
import sys
import os
from torchinfo import summary

# Ajouter le chemin pour importer ResNet18
sys.path.append(os.path.abspath("../LAB1"))
from resnet import ResNet18

def calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True):
    # Créer le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet18().to(device)
    
    # Afficher les statistiques du modèle
    stats = summary(model, (1, 3, 32, 32), verbose=0)
    
    total_params = stats.total_params
    total_macs = stats.total_mult_adds  # Nombre d'opérations MAC (approximation des FLOPs)
    
    # Convertir MAC en FLOPs (environ 2 FLOPs par MAC)
    total_flops = total_macs * 2
    
    # Calculer le score
    p_s = structured_pruning
    p_u = global_pruning
    q_w = 16 if is_half_precision else 32  # 16 bits pour half precision, 32 pour full
    
    # On suppose q_a = 32 bits pour les activations
    max_q = max(q_w, 32)
    
    w = total_params
    f = total_flops
    
    param_term = (1 - (p_s + p_u)) * (q_w/32) * w / (5.6e6)
    ops_term = (1 - p_s) * (max_q/32) * f / (8.3e8)
    
    score = param_term + ops_term
    
    print(f"Nombre total de paramètres: {total_params:,}")
    print(f"Nombre total d'opérations (FLOPs): {total_flops:,}")
    print(f"Score calculé: {score:.4f}")
    
    return score

# Calculer le score avec vos paramètres
score = calculate_score(structured_pruning=0, global_pruning=0, is_half_precision=False)

Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 3.3337
